In [ ]:
import os
import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

%run /work/greenhouse-simulator-2/greenhouse/structure/structure.ipynb import Structure

from helpers.types import *
from helpers.data_prep import *
from helpers.visualization import *
from helpers.solar_conversions import *

In [ ]:
class NonSolar:
    def __init__(self):
        self.transparency = 0.8
        self.efficiency = 0
        self.configuration = "static"

In [ ]:
class SolarBrite:
    def __init__(self, transparency=0.44):
        self.transparency_limits = [0.44, 0.82]
        assert transparency >= self.transparency_limits[0] and transparency <= self.transparency_limits[1], f"SolarBrite transparency ({transparency}) is not within limits {self.transparency_limits}"
        self.transparency = transparency
        self.efficiency = 0.22 # As portion of non-transmitted sunlight
        self.configuration = "static"

In [ ]:
class Insolight2:
    def __init__(self):
        self.transparency_limits = [0.2, 0.78]
        self.transparency = 0.4
        self.efficiency = 0.2 # As portion of non-transmitted sunlight
        self.configuration = "dynamic"

In [ ]:
class SolarPanel:
    def __init__(self, time_period: s, photoperiod: h, target_DLI: mol_per_m2_day, irradiated_area: m2, roof_panel_type):
        self.time_period: s = time_period
        self.photoperiod: h = photoperiod
        self.target_DLI: dli = target_DLI
        self.irradiated_area: m2 = irradiated_area

        supported_roof_panel_types = ["solarbrite", "insolight", "polycarbonate"]
        assert roof_panel_type in supported_roof_panel_types, f"roof_panel_type must to be in {supported_roof_panel_types}"

        if roof_panel_type == "solarbrite":
            self.panel = SolarBrite()
        elif roof_panel_type == "insolight":
            self.panel = Insolight2()
        elif roof_panel_type == "polycarbonate":
            self.panel = NonSolar()


    def distribute_irradiance_static(self, irradiance: W_per_m2) -> [W_per_m2, W_per_m2]:
        transmitted_irradiance: W_per_m2 = irradiance * self.panel.transparency
        irradiance_on_panels: W_per_m2 = irradiance * (1 - self.panel.transparency)
        return transmitted_irradiance, irradiance_on_panels, self.panel.transparency


    def distribute_irradiance_dynamic(self, irradiance: W_per_m2) -> [W_per_m2, W_per_m2]:
        PPFD: mol_per_m2_s = irradiance_to_PPFD(irradiance)
        projected_DLI: mol_per_m2_day = PPFD_to_projected_DLI(PPFD, self.photoperiod)

        transparency = self.target_DLI / (projected_DLI + 1e-10) # add a small number to avoid ZeroDivisionError

        # Keep transparency within valid limits
        if transparency > self.panel.transparency_limits[1]:
            transparency = self.panel.transparency_limits[1]

        if transparency < self.panel.transparency_limits[0]:
            transparency = self.panel.transparency_limits[0]
        
        transmitted_irradiance: W_per_m2 = irradiance * transparency
        irradiance_on_panels: W_per_m2 = irradiance * (1 - transparency)

        return transmitted_irradiance, irradiance_on_panels, transparency


    def run(self, irradiance_on_solar_panel: W_per_m2) -> [W, W, W_per_m2]:
        if self.panel.configuration == "static":
            transmitted_irradiance, irradiance_on_panels, transparency = self.distribute_irradiance_static(irradiance_on_solar_panel)
        elif self.panel.configuration == "dynamic":
            transmitted_irradiance, irradiance_on_panels, transparency = self.distribute_irradiance_dynamic(irradiance_on_solar_panel)

        solar_power_transmitted: W = transmitted_irradiance * self.irradiated_area
        power_captured: W = irradiance_on_panels * self.irradiated_area
        electrical_power_generated: W = self.panel.efficiency * power_captured

        transmitted_PPFD: umol_per_m2_s = irradiance_to_PPFD(transmitted_irradiance)
        target_PPFD: umol_per_m2_s = projected_DLI_to_PPFD(self.target_DLI, self.photoperiod)

        wasted_PPFD: umol_per_m2_s = 0
        if transmitted_PPFD > target_PPFD:
            wasted_PPFD = transmitted_PPFD - target_PPFD

        return electrical_power_generated, solar_power_transmitted, transmitted_irradiance, transparency, wasted_PPFD

## Visualize

### Static transparency: SolarBrite

In [ ]:
if not os.environ.get("USING_RUN"):
    resample_period = "15min"
    time_period = pd.to_timedelta(resample_period).total_seconds()

    structure = Structure()
    panel = SolarPanel(
        time_period = time_period,
        photoperiod = 16,
        target_DLI = 13,
        irradiated_area = 15, 
        roof_panel_type = "solarbrite"
    )

In [ ]:
if not os.environ.get("USING_RUN"):
    df = get_weather_data(date_from="2020-05-02", date_to="2020-05-03", resample_period=resample_period)

    for i, row in df.iterrows():
        results: [W, W_per_m2] = structure.get_irradiance_by_panel_type(i, row.solarradiation)
        solar_power_on_nonsolar_panels, irradiance_on_solar_panels = results

        results = panel.run(irradiance_on_solar_panels)
        electrical_power_generated, solar_power_transmitted, transmitted_irradiance, transparency, wasted_PPFD = results

        df.loc[i, "electrical_power_generated_W"] = electrical_power_generated
        df.loc[i, "solar_power_transmitted_W"] = solar_power_transmitted
        df.loc[i, "transmitted_irradiance_W_per_m2"] = transmitted_irradiance
        df.loc[i, "wasted_PPFD"] = wasted_PPFD


    total_power_generated: kWh = df["electrical_power_generated_W"].sum() * time_period / 1000 / 3600
    print(f"Total power generated: {round(total_power_generated, 2)} kWh")

In [ ]:
plots_2 = None
if not os.environ.get("USING_RUN"):
    plots_2 = plot_multiline(
        df, 
        ["electrical_power_generated_W", "solar_power_transmitted_W"], 
        width=900, 
        height=235, 
        title="", 
        y_label="Watts", 
        legend_label="", 
        date_format=""
    ) & \
    plot_multiline(
        df, 
        ["transmitted_irradiance_W_per_m2"], 
        width=900, 
        height=235, 
        title="", 
        y_label="Watts per m2", 
        legend_label="", 
        date_format=""
    ) & \
    plot_multiline(
        df, 
        ["wasted_PPFD"], 
        width=900, 
        height=235, 
        title="", 
        y_label="umol_per_m2_s", 
        legend_label="", 
        date_format=""
    )
plots_2

### Dynamic transparency: Insolight

In [ ]:
if not os.environ.get("USING_RUN"):
    structure_2 = Structure()
    panel_2 = SolarPanel(
        time_period = time_period,
        photoperiod = 16,
        target_DLI = 13,
        irradiated_area = 15, 
        manufacturer = "insolight"
    )

In [ ]:
if not os.environ.get("USING_RUN"):
    df = get_weather_data(date_from="2020-05-02", date_to="2020-05-03", resample_period=resample_period)

    for i, row in df.iterrows():
        results: [W, W_per_m2] = structure_2.get_irradiance_by_panel_type(i, row.solarradiation)
        solar_power_on_nonsolar_panels, irradiance_on_solar_panels = results

        results = panel_2.run(irradiance_on_solar_panels)
        electrical_power_generated, solar_power_transmitted, transmitted_irradiance, transparency, wasted_PPFD = results


        df.loc[i, "electrical_power_generated_W"] = electrical_power_generated
        df.loc[i, "solar_power_transmitted_W"] = solar_power_transmitted
        df.loc[i, "transmitted_irradiance_W_per_m2"] = transmitted_irradiance
        df.loc[i, "transparency"] = transparency
        df.loc[i, "wasted_PPFD"] = wasted_PPFD

    total_power_generated: kWh = df["electrical_power_generated_W"].sum() * time_period / 1000 / 3600
    print(f"Total power generated: {round(total_power_generated, 2)} kWh")

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    plots = plot_multiline(
        df, 
        ["electrical_power_generated_W", "solar_power_transmitted_W"], 
        width=900, 
        height=235, 
        title="", 
        y_label="Watts", 
        legend_label="", 
        date_format=""
    ) & \
    plot_multiline_dual_y(
        df, 
        ["transmitted_irradiance_W_per_m2"],
        ["transparency"],
        width=900, 
        height=235, 
        title="", 
        y_labels=["Watts per m2", "transparency"], 
        legend_label="", 
        date_format=""
    ) & \
    plot_multiline(
        df, 
        ["wasted_PPFD"], 
        width=900, 
        height=235, 
        title="", 
        y_label="umol_per_m2_s", 
        legend_label="", 
        date_format=""
    )
plots

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>